In [23]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

# ============================
# 1) CTI — contract_tree_information
# ============================
print("\n=== CTI HEADERS ===")
q_cti = text("SELECT * FROM masterdatabase.contract_tree_information LIMIT 0;")
df_cti = pd.read_sql(q_cti, engine)
print(list(df_cti.columns))


# ============================
# 2) FPI — farmer_personal_information
# ============================
print("\n=== FPI HEADERS ===")
q_fpi = text("SELECT * FROM masterdatabase.farmer_personal_information LIMIT 0;")
df_fpi = pd.read_sql(q_fpi, engine)
print(list(df_fpi.columns))


# ============================
# 3) CA — contract_allocations
# ============================
print("\n=== CA HEADERS ===")
q_ca = text("SELECT * FROM masterdatabase.contract_allocation LIMIT 0;")
df_ca = pd.read_sql(q_ca, engine)
print(list(df_ca.columns))


💻 Conectado a la base de datos helloworldtree

=== CTI HEADERS ===
['contract_amendment', 'contract_code', 'etp_year', 'harvest_year', 'land_location', 'latitude', 'longitude', 'planted', 'planting_obligation', 'planting_year', 'region', 'species', 'strain', 'trees_contract', 'loaded_at', 'planting_date', 'status', 'Filter', 'land_location_pro', 'etp_year_backup']

=== FPI HEADERS ===
['farmer_number', 'representative', 'phone', 'email', 'address', 'shipping_address', 'contract_codes', 'contract_name']

=== CA HEADERS ===
['contract_code', 'usa_allocation_pct', 'usa_trees_contracted', 'usa_trees_planted', 'canada_allocation_pct', 'canada_2017_trees', 'canada_2017_allocation_pct', 'canada_trees_contracted', 'total_can_allocation', 'loaded_at', 'etp_type', 'contracted_cop', 'planted_cop', 'contracted_etp', 'planted_etp', 'surviving_etp', 'surviving_cop']


In [24]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

q = text("""
    SELECT
        -- CTI
        cti.contract_code,
        cti.etp_year,
        cti.planting_year,
        cti.status,

        -- FPI (array)
        fpi.representative,
        fpi.farmer_number,
        fpi.contract_name,

        -- CA
        ca.canada_trees_contracted AS can_trees_allocated,
        ca.canada_2017_trees      AS can_2017_trees_allocation,
        ca.usa_trees_contracted   AS usa_trees_allocated

    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.farmer_personal_information fpi
        ON cti.contract_code = ANY(fpi.contract_codes)
    LEFT JOIN masterdatabase.contract_allocation ca
        ON cti.contract_code = ca.contract_code
    ;
""")

df = pd.read_sql(q, engine)
df.head()

print(df.columns)


💻 Conectado a la base de datos helloworldtree
Index(['contract_code', 'etp_year', 'planting_year', 'status',
       'representative', 'farmer_number', 'contract_name',
       'can_trees_allocated', 'can_2017_trees_allocation',
       'usa_trees_allocated'],
      dtype='object')


In [18]:
column_order = {
    1:  "contract_code",
    2:  "etp_year",
    3:  "planting_year",

    7:  "representative",
    8:  "farmer_number",
    9:  "contract_name",
    10:  "status",

    4:  "can_trees_allocated",
    5:  "can_2017_trees_allocation",
    6: "usa_trees_allocated"
}
df = df[list(column_order.values())]


In [25]:
column_order_pretty = {
    1:  ("contract_code",              "Contract Code"),
    2:  ("etp_year",                   "ETP Year"),
    3:  ("planting_year",              "Planting Year"),

    7:  ("representative",             "Representative"),
    8:  ("farmer_number",              "Farmer#"),
    9:  ("contract_name",              "Contract Name"),
    10:  ("status",                     "Status"),

    4:  ("can_trees_allocated",        "Can/trees allocated"),
    5:  ("can_2017_trees_allocation",  "Can 2017/trees allocation"),
    6: ("usa_trees_allocated",        "USA/trees allocated")
}

print("COLUMNS IN DF RIGHT NOW:")
print(df.columns.tolist())


COLUMNS IN DF RIGHT NOW:
['contract_code', 'etp_year', 'planting_year', 'status', 'representative', 'farmer_number', 'contract_name', 'can_trees_allocated', 'can_2017_trees_allocation', 'usa_trees_allocated']


In [26]:
# 1) seleccionar usando los nombres originales
df = df[[v[0] for v in column_order_pretty.values()]]

# 2) renombrar
df.columns = [v[1] for v in column_order_pretty.values()]

print(df.columns)


Index(['Contract Code', 'ETP Year', 'Planting Year', 'Representative',
       'Farmer#', 'Contract Name', 'Status', 'Can/trees allocated',
       'Can 2017/trees allocation', 'USA/trees allocated'],
      dtype='object')
